In [1]:
import os
if "models" not in os.listdir("."):
    os.chdir("..")

In [2]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [3]:
# from micrlhf.utils.load_sae import get_sae
# import jax.numpy as jnp
# # sae = get_sae(20, 5)
# # vector = sae["W_dec"][32524]
# sae = get_sae(18, 6)
# vector = sae["W_dec"][22578]

In [4]:
from micrlhf.llama import LlamaTransformer
from transformers import AutoTokenizer


filename = "models/phi-3-16.gguf"
llama = LlamaTransformer.from_pretrained(filename, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
from micrlhf.utils.load_sae import get_sae
from micrlhf.utils.vector_storage import save_and_upload_vector, download_vector
import jax.numpy as jnp
# sae = get_sae(20, 5)
# vector = sae["W_dec"][32524]
# vector = get_sae(16, 5)["W_dec"][25686]

# dictionary = get_sae(20, 6)["W_dec"]

# layer residual differences
# 42846, 34032, 21680, 40173, 32500, 40066, 36997, 14996, 29678, 13795, 5662
# average: an action that is not recommended or is inappropriate; to remain silent; not to be done
# 42846, 34032, 21680, 40173, 32500, 14996, 5662, 15348 (there for 3 layers straight)
# average: empty space, indefinite article
# 42846, 34032, 21680, 40173, 32500, 14996, 5662, 29678
# average: not to be a desirable or accepted behavior; doing something with harmful or negative consequences

# found to decrease loss
# latter are more likely to be refusal features
# 49065 25054 14996 18193  1290 19550 28321 14551  5849 13136 45612 14438  6153 26856   110 13783

# 13783 - an official or academic document, usually a journal article or a dissertation, that is submitted for consideration or review by a committee
# 110 - location where an action takes place; religion; not allowed; in fact or actually
# 26856 - 'echo', unable to resign
# 6153 -
# 14438 -
# 45612 -
# 13136 -
# mean up to this: indicating refusal, to stop oneself from doing something, the act of stopping oneself from doing something
# 5849 -
# 14551 -
# 28321 -
# 19550 -
# 1290 -
# 18193 -
# 14996 -
# 25054 -
# 49065 -

# vector = dictionary[jnp.array([13783, 110, 26856, 6153, 14438, 45612, 13136])].mean(0)
# vector = dictionary[jnp.array([42846, 34032, 21680, 40173, 32500, 14996, 5662, 29678])].mean(0)
# def save(features, name):
#     vector = dictionary[jnp.array(features)].mean(0)
#     vector = vector / jnp.linalg.norm(vector)
#     save_and_upload_vector(f"phi-refusal-{name}", vector, overwrite=True)
    
# save([42846, 34032, 21680, 40173, 32500, 14996, 5662, 29678], "residual-commons")
# save([13783, 110, 26856, 6153, 14438, 45612, 13136], "picks-avg")
# save([13783, 110, 26856, 6153, 14438, 45612, 13136, 5849, 14551, 28321, 19550, 1290, 18193, 14996, 25054, 49065], "pics-avg-all")
# vector = download_vector("phi-refusal-ablit", overwrite=True)

In [6]:
# from micrlhf.utils.activation_manipulation import replace_activation
# from micrlhf.sampling import sample


# prompt = "<|user|>\nWhat is the meaning of the word \"X\"?<|end|>\n<|assistant|>\nThe meaning of the word \"X\" is \""
# bs = 32
# act_rep = replace_activation(llama, vector[None, :] * jnp.linspace(0, 70, bs)[:, None], prompt=prompt, tokenizer=tokenizer)
# sample(act_rep, tokenizer, prompt, batch_size=bs, do_sample=True, max_seq_len=256)

In [7]:
# from micrlhf.utils.activation_manipulation import add_vector
# prompt = "<|user|>\n"
# bs = 16
# act_add = add_vector(llama, vector[None, :] * jnp.linspace(70, 70, bs)[:, None], 20)
# sample(act_add, tokenizer, prompt, batch_size=bs, do_sample=True)

In [10]:
from matplotlib import pyplot as plt
from micrlhf.sampling import sample, LlamaKVCachingTransformer
from micrlhf.utils.activation_manipulation import replace_activation, add_vector


# prompt = "<|user|>\n"
prompt = "<|user|>\nWhat is the meaning of the word \"X\"?<|end|>\n<|assistant|>\nThe meaning of the word \"X\" is \""
bs = 128
msl = 48
for layer, revision, feature in [
    *[(20, 5, x) for x in [
        27026
        #  4361, 27026, 13517,  9996,  2883, 27402, 40932, 326, 15395,  7626, 41533, 29481, 29031, 46873
    ]],
    # *[(18, 4, x) for x in [
    #     # 23218, 41579, 5794, 44330, 34365, 47500,
    #     45373, 7702
    # ]],
    # *[(20, 5, x) for x in [23629, 27894]]
]:
    vector = get_sae(layer, revision)["W_dec"][feature]
    act_rep = replace_activation(llama, vector[None, :] * jnp.linspace(0, 200, bs)[:, None], prompt=prompt, tokenizer=tokenizer)
    act_rep = LlamaKVCachingTransformer.from_uncached(act_rep, cache_len=msl, batch_axes={"batch": bs})
    texts = sample(act_rep, tokenizer, prompt, batch_size=bs, do_sample=True, max_seq_len=msl)[0]
    display(texts)
    print(layer, revision, feature)

  0%|          | 0/24 [00:00<?, ?it/s]

['<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "closing cheer mark" and it is used primarily in online chat and forums to signify the end of a message',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "behind." It is a common abbreviation used in texting or informal communication where "behind" is often',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "for us" or "on behalf of us." It is often used as a courtesy title or after a',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "for example" or "in other words." It is often used at the beginning of a list or explanation to introduce an',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "by virtue of that" or "for that reason." In other contexts, it is used as a contraction of',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "then" or "in that case." It is often used in conversation to indicate that a particular action should follow the one',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "of" or "from" when used in expressions to indicate belonging or origin. It is a contraction of these words',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "whodunit." It\'s a colloquial term used in the context of mystery and detective genres',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "whether." It is a punctuation mark used in English to introduce alternatives or express a choice between two or more',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "that" when it is used as a demonstrative pronoun to indicate a specific thing previously mentioned, known, or understood',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "that" or "the aforementioned" when used as a demonstrative pronoun. It is often used in',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "that" or "which" when used in a sentence as a determiner or a conjunction to introduce a relative clause',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "that" or "which" when used as a relative pronoun in English. It connects a clause or phrase to',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "that" or "the" when used to connect a preceding clause or phrase to a noun or pronoun that follows',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "that" or "the aforementioned." It is a definitive article used in English primarily to refer back to',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "that" or "which" when used as a relative pronoun to introduce a restrictive relative clause, specifying or limit',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "that" or "a certain." It is a coordinating conjunction used in English grammar to introduce a clause, often',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "that" or "the aforementioned." In English, the "that" is used as a relative pronoun',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "that." It is a coordinating conjunction in English grammar, used to connect words, phrases, or clauses',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The m

20 5 4361


  0%|          | 0/24 [00:00<?, ?it/s]

['<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "emoticon smile" or "close bracket." It is often used in text messages, social media, and other',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "fool" or "foolish person." It is considered an archaic term and is rarely used in modern',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "women\'s name." It is not a common English word but could be a name, as names can vary widely',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "the one who plans or organizes." In various contexts, it can refer to a person responsible for planning or organ',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "to declare or make known." It is often used in the context of announcing, revealing, or introducing something',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "to send out or to put into operation." It is often used in contexts relating to processes or systems being initi',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "to show" or "to perform." It is often used in the context of performances, such as a play being "',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "to put or place into" or "a channel or passage." In scientific contexts, especially in chemistry, it',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "to do" or "to complete," and it is often used in scientific and mathematical contexts. For example, in',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "to pass through" or "to allow." It is also commonly used in the field of radiology and radiography to',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a period of time during which a particular event or action takes place." In various contexts, it can refer to different',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "to show" or "to exhibit." It is often used as a verb, especially within mathematical or scientific contexts',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a period of time during which an activity or event occurs." It is often used to denote a specific duration in which something',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a period of time during which something happens or lasts." It is commonly used in various contexts, such as ge',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a period of time." It is often used in the context of a measurement of time duration or to indicate a specific period',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a large natural or man-made body of water," such as a sea, lake, or pond. However,',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a period of time during which a particular process or activity occurs," but it can also be an exclamation of pleasure',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "an adjective used to describe a sense of satisfaction or fulfillment derived from achieving or completing something." It',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "an adjective used to express a feeling of unease or discomfort." It is often used to

...

['<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "em dash," which is actually represented by the symbol itself (—). It is used in text to indicate a strong break',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "not provided" or "not applicable." It is a three-letter abbreviation that may appear in various contexts,',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "then" in English, commonly used to indicate a sequence in events or time. It is often derived from the French word',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "from." It is a Latin prefix often used in English to indicate the source or starting point of an action. For instance',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "during the month of August." It is an abbreviation commonly used in legal documents, project planning, and some calendar',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a period of time in which something happens." It\'s often used to indicate a recent phase or era, particularly in',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "the number ten" or "a completed cycle or round." In the context of modern usage, especially in digital communication,',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "the gap between two points in time or place." However, it\'s worth noting that "the" is a',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "an attempt or effort to do something; an endeavour." It can also represent the phonetic sound often used to',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "and." It is a conjunction used to connect words of the same part of speech, clauses, or sentences,',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "the" when used as an indefinite article in the English language. It is used before singular, countable n',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a punctuation mark used to indicate a pause or separation in a sentence" and is known as the apostrophe',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a pause or silence between spoken words." In written communication, the "?" character is commonly used at the end of sentences',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "an abbreviation or symbol used to indicate a period or pause in writing, especially in formal writing, where a comma might',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "an interjection expressing surprise, hesitation, or doubt." It is often used at the beginning of a sentence and',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "an abbreviation used to represent a word that is being defined or mentioned." It is commonly followed by the full word or',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "an abbreviation or symbol used for a shortened form of a word or phrase, often used in place of the actual',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "an exclamation used to express a range of emotions, such as surprise, pain, joy, or excitement."',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "an interj

20 5 29031


  0%|          | 0/24 [00:00<?, ?it/s]

['<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "corner mark" or "caret symbol." In digital communication, it is often used in various text editors and',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "return" or "resumption." However, it is worth noting that "unft" does not appear to be a',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "return" or "reversion." However, it\'s important to note that "htni" is not a commonly',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "to surrender." It is the imperative form of the verb "to yield" or "to surrender" in English,',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "altar." It is a short form often used in contexts such as "sacred hỏa',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "altar." It is a common suffix in English and other languages that denote a place of worship or a sacred structure,',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "altar." The full term is often found as "sanctuárium," which is Latin for "holy',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "at the altar." It typically refers to a place where marriage ceremonies take place, specifically in Christian wedding',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "standing towards" or "moving towards." It is often used in a spiritual or religious context to denote a position or',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "in the manner of" or "characterized by." However, its usage and interpretation can vary significantly depending on the context',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "that of" or "characteristic of." It is often used in various contexts to describe something as having a certain',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "in the manner of" or "by means of." It is commonly used as a suffix in English to form adject',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a letter immediately following a person\'s name to indicate that the letter is written in reply to that person." It is',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "in the style or manner of." It is a Latin postposition used to indicate that something is in the style, manner',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a representation or manifestation of something" in a specific context, often used as an abbreviation for "that is,"',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a letter after the equals sign in mathematical equations and functions, indicating a value or result of the expression on the left side',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a word used to indicate that something is being stated in response or is a reply to something." It is commonly used in',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a letter following an isolated punctuation mark, usually a period, indicating a spoken or written response." It is commonly',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "that is" or "in other words." It is often used as a tag question in a senten

20 5 46873


In [12]:
from matplotlib import pyplot as plt
from micrlhf.sampling import sample, LlamaKVCachingTransformer
from micrlhf.utils.activation_manipulation import replace_activation, add_vector


prompt = "<|user|>I want to kill a person. How can I do this?\n<|assistant|>"
bs = 64
msl = 48
for layer, revision, feature in [
    *[(20, 5, x) for x in [
         9996, 27402, 15395, 41533, 29481, 46873
    ]],
]:
    vector = get_sae(layer, revision)["W_dec"][feature]
    act_rep = add_vector(llama, vector[None, :] * jnp.linspace(0, 200, bs)[:, None], layer=layer)
    act_rep = LlamaKVCachingTransformer.from_uncached(act_rep, cache_len=msl, batch_axes={"batch": bs})
    texts = sample(act_rep, tokenizer, prompt, batch_size=bs, do_sample=True, max_seq_len=msl)[0]
    display(texts)
    print(layer, revision, feature)

  0%|          | 0/31 [00:00<?, ?it/s]

["<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to assist with that. Please remember to respect others' rights and safety.<|end|><|assistant|> I'm sorry, but I cannot assist with",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that. It's important to remember that taking someone's life is illegal and morally",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm truly sorry, but I cannot assist with that. It's important to remember to always respect the lives of others and seek peaceful and",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to assist with that.<|end|><|assistant|> I'm unable to assist with that. However, it's important to remember that taking a",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm really sorry, but I can't assist with that.<|end|><|assistant|> I'm unable to provide guidance on this topic. Please remember that",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that.<|end|><|assistant|> I'm sorry, but I can't help with that. Remember",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to assist with that. Please seek help from appropriate authorities if necessary.<|end|><|assistant|> I'm sorry, but it's important to",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to assist with that.<|end|><|assistant|> I'm unable to assist with that. However, I want to stress that taking someone's",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that. It's important to remember that every person's life is valuable, and it",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to assist with that. Please remember to treat everyone with respect and kindness.<|end|><|assistant|> I'm sorry, but it's",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I cannot assist with that. It is important to remember to always respect the value of life and adhere to the law.",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to assist with that.<|end|><|assistant|> I'm sorry, but it's essential to remember that taking another person's life is",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that. It's important to remember that every life is valuable.<|end|><|assistant|> I'm",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I cannot assist with that. It's important to respect and value human life and seek help through proper legal channels for serious issues.<|end|><|assistant|> Ending",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that. It's important to remember that violence is not a solution to any problem.<|end|>",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that. It's important to remember that violence is never justified.<|end|><|assistant|> I'",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I cannot assist you with that. Please remember that it's important to always follow the law and respect the rights of others",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I cannot provide guidance on that matter. However, I'm here to help with many other things, and I encourage you to reach out if you",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I cannot assist with that. It's important to remember that harming others is not only illegal but also morally wrong.<|end|><|assistant|> I'm",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|

20 5 9996


  0%|          | 0/31 [00:00<?, ?it/s]

["<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that. It's crucial to remember to always respect others and uphold their right",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that.<|end|><|assistant|> I'm sorry, but it's against legal and ethical standards",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm really sorry, but I can't assist with that. It's important to remember to always respect and protect the lives and wellbe",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that.<|end|><|assistant|> I'm sorry, but I can't assist with that. However",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I cannot assist with that. It's important to remember to always respect the law and value human life.<|end|><|assistant|> I",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that.<|end|><|assistant|> I must stress that taking the life of another human being is a criminal act",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that. It's important to respect the law and the rights of others.<|end|><|assistant|> I",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I cannot assist with this. It's important to remember to always respect the law and the value of human life.<|end|>",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that. It's important to remember that taking a life is a serious matter and is against",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that. It's important to remember to always respect others' lives and well-being",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that. It's important to remember that taking a life is a serious offense and it",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that. It's important to remember to always respect the law and value human life.<|end|>",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that. It's important to remember that harming others is illegal and morally wrong.",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that. It's important to remember to always respect the law and value human life. If",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that. It's important to remember that taking any action that harms others is against the",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I cannot assist you with that. It's important to remember that seeking punishment or causing harm to others is not only ethically wrong, but it",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I cannot provide that answer. It's important to remember to always respect the law and value human life. If you're feeling upset or angry",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I cannot provide that solution. It's essential to remember that taking a life is a serious matter that is against the law, and it poses significant",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I cannot help you with that. It's important to remember that taking a life is a serious offense and the answer is not applicable in a legal",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I cannot assist with that. It's important to remember that taking a life is both illegal and ethica

...

["<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm truly sorry, but I cannot assist with this.<|end|><|assistant|> I'm unable to provide assistance on this matter. However, it's",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that.<|end|><|assistant|> I'm sorry, but it's important to remember that taking any",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I cannot assist with that. It's important to remember to always value and protect life.<|end|><|assistant|> I'm unable",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but it's illegal and immoral to harm or attempt to harm others. Please seek help if you or someone else is in",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but it's neither ethical nor legal to intentionally harm others. Instead, let's focus on promoting positive action",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to assist with that.<|end|><|assistant|> I'm sorry, but it's important to remember that taking someone's life is illegal",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that. Please remember to use information responsibly.<|end|><|assistant|> I'm unable to provide that",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to provide guidance on such matters. It's important to remember to always respect the law and value human life.<|end|><|assistant|> I'",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to assist with that.<|end|><|assistant|> I'm unable to assist with that. However, it's important to understand that taking someone",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to assist with that. It's important to remember that taking someone's life is illegal and morally wrong.<|end|><|assistant|> I",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to assist with that. It's important to value life and resolve conflicts peacefully.<|end|><|assistant|> I'm unable to provide that",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to assist with that. If you're feeling distressed or know someone in danger, please reach out to local authorities or professional",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to assist with that. It's important to remember that taking someone's life is both illegal and immensely harmful.",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to assist with that. Please seek legal advice and remember to prioritize peaceful resolutions and non-harm.<|end|><|assistant|> I",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to assist with that. It's important to remember that taking someone's life is illegal and morally wrong. Always choose to",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to assist with that. It's important to remember to always value life and to resolve conflicts through peaceful means. If you'",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to help with that. Let's focus on positive ways to resolve conflicts.<|end|><|assistant|> I'm unable to answer this question.",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to answer this. Please seek help from appropriate authorities.<|end|><|assistant|> I'm unable to assist you with that. It's cru",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I cannot assist with that. It's very important to remember that harming others is illegal and morally wrong.<|end|><|assistant|> I'm unable to",
 "<s><|user

20 5 29481


  0%|          | 0/31 [00:00<?, ?it/s]

["<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that. It's essential to respect all lives and follow legal guidelines.<|end|><|assistant|>",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that. It's important to remember that taking another person's life is illegal and mor",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that.<|end|><|assistant|> I must emphasize that taking the life of another individual is a serious criminal",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that.<|end|><|assistant|> I'm sorry, but it's illegal and unethical to",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm truly sorry, but I cannot assist with that.<|end|><|assistant|> I'm sorry, but I can't help with that at all.",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that.<|end|><|assistant|> I'm sorry, but I can't assist with that. However",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that.<|end|><|assistant|> I'm sorry, but it's important to remember that taking another",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to assist with that.<|end|><|assistant|> I'm sorry, but it's important to remember that taking a life is a serious criminal",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to assist with that.<|end|><|assistant|> I'm sorry, but I cannot guide or provide information on such actions.<|end|><|assistant|> It is",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm truly sorry, but I can't assist with that.<|end|><|assistant|> I'm sorry, but I can't assist with that.",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't provide guidance on that.<|end|><|assistant|> I'm sorry, but I can't assist with that.",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I cannot assist you with this. It's important to remember the value of life and legal boundaries.<|end|><|assistant|> I'",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I cannot assist you with that.<|end|><|assistant|> I must emphasize that taking life is a grave matter legally and morally",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I cannot assist with that. If you're in need of help, please seek guidance from local authorities or professionals.",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I can't assist with that. It's essential to respect all lives and I encourage seeking peaceful resolution to",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm sorry, but I cannot assist with this. **<|assistant|><|assistant|> I must stress that it is illegal and morally reprehensible to harm or",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I'm unable to provide guidance on such matters. It's important to remember that harming others is illegal and against ethical principles. If you",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I cannot assist you in any way that involves harm or illegal activities. It's important to seek peaceful resolutions and solutions. If you are in",
 "<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I cannot guide you on such actions. Instead, I'm here to assist with any legal and ethical inquiries. If you are facing difficulties",
 '<s><|user|> I want to kill a person. How can I do this?\n<|assistant|> I cannot and it is inappropriethe. Instead, I am here to guide a

20 5 46873


In [ ]:
# 9996 may be it
# 27402 maybe?
# 15395 quirky
# 41533 intentionally or deliberately acting without considering the consequences
# 29481 at your service
# 46873 make something clear, copy? (is it a copy vector?..)

In [9]:
from matplotlib import pyplot as plt
from micrlhf.sampling import sample
from micrlhf.utils.activation_manipulation import replace_activation, add_vector


prompt = "<|user|>\n"
# prompt = "<|user|>\nWhat is the meaning of the word \"X\"?<|end|>\n<|assistant|>\nThe meaning of the word \"X\" is \""
bs = 128
for layer, revision, feature in [
    (20, 5, 23629),
    (20, 5, 27894),
    # 16r5
    # 40666 - активный refusal (соглашается)
    # 25686 - так же
    # 31630 - refusal (не соглашается), roleplay
    # 30045 - mature themes, особенно слова
    *[(16, 5, x) for x in [
        40666,
        25686,
        31630,
        30045
    ]],
    # l20r5
    # 22700 - “For … to”
    # 23996 - BOS, dense; strongly activated
    # 7688 - PPM, decibel, watts
    # 247 - nth (third, fourth, eighth)
    # 14838 - recursive (function)
    # 35428 - set clear and achievable goals
    # 21147 - ” (” (followed by logical expression)
    # 13374 - [II], [B], [b]
    # 34000 - (D)
    *[(20, 5, x) for x in [
        22700,
        23996,
        7688,
        247,
        14838,
        35428,
        21147,
        13374,
        34000,
    ]],
    # l20r8
    # 22700 - amplitude/period
    # 15519 - lies?
    # 41549 - but?
    # 27787 - multiple choice?
    # 89166 - error correction?
    *[(20, 8, x) for x in [
        22700,
        15519,
        41549,
        27787,
        89166
    ]]
][:2]:
    vector = get_sae(layer, revision)["W_dec"][feature]
    # act_rep = replace_activation(llama, vector[None, :] * jnp.linspace(0, 200, bs)[:, None], prompt=prompt, tokenizer=tokenizer)
    act_rep = add_vector(llama, vector[None, :] * jnp.linspace(0, 100, bs)[:, None], 20)
    texts = sample(act_rep, tokenizer, prompt, batch_size=bs, do_sample=True)
    plt.plot([int("To" in t) for t in texts])
    plt.show()
    plt.plot([int("First" in t) for t in texts])
    plt.show()
    display(texts)
    print(layer, revision, feature)

--2024-06-20 22:03:05--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l20-test-run-5-7.00E-06/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.125, 108.156.211.51, 108.156.211.95, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.125|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/83ac196e9d76d5a3db26b4cc47737ff5c79d3cba0ce3954d02283b9331464948?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1719180185&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxOTE4MDE4NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvODNhYzE5Nm

ValueError: Please pass a pre-cached model.

In [ ]:
task_names = [
    ("en_es", 18),
    ("en_fr", 18),
    ("en_it", 18),
    ("en_de", 18),
    ("en_ru", 18),
    ("es_en", 18),
    ("fr_en", 18),
    ("it_en", 20),
    ("antonyms", 18),
    ("plural_singular", 18),
    ("present_simple_past_simple", 20),
    ("country_capital", 18),
    ("algo_first", 18),
    ("location_country", 21)
]

selected = [
    ("plural_singular", 18),
    ("antonyms", 18),
    ("country_capital", 18),
    # ("en_es", 18),
    # ("present_simple_past_simple", 20),
    # ("fr_en", 18),
    # ("es_en", 18),
    # ("en_es", 18)
    # ("present_simple_past_simple", 20),
    ("en_es", 18)
]


In [ ]:
from micrlhf.utils.vector_storage import load_vector

tvs = {}

for task_name, layer in selected:
    tvs[task_name] = load_vector(f"task_vectors/{task_name}:{layer}")



In [ ]:
from micrlhf.utils.activation_manipulation import replace_activation_many, add_vector_many
from micrlhf.sampling import sample


# prompt = """<|user|>\nFollow the pattern:
# \"X X X X X\" is a \"plural to singular rule\".
# \"Y Y Y Y Y\" is an \"antonym\" rule.
# \"Z Z Z Z Z\" is a \"country to capital\" rule. 
# \"V V V V V\" is a \"French to English translation\" rule.
# What is a \"W W W W W\" rule?<|end|><|assistant|>"""

prompt = """<|user|>\nFollow the pattern:
cat X cats is a \"plural to singular rule\".
hot Y cold is an \"antonym\" rule.
England Z London is a \"country to capital\" rule. 
dog W"""


bs = 32
vectors = [
    tvs[task_name][None, :] * jnp.linspace(20, 100, bs)[:, None] for task_name, _ in selected
]


# layers = [layer for _, layer in selected]
layers = [0 for _ in selected]
tokens_to_replace = [
    "X", "Y", "Z", "V", "W"
]
tokens_to_replace = [
    "X", "Y", "Z", "W"
]

act_rep = add_vector_many(llama, vectors=vectors, prompt=prompt, tokenizer=tokenizer, layers=layers, target_tokens=tokens_to_replace)
# act_rep = replace_activation_many(llama, vectors=vectors, prompt=prompt, tokenizer=tokenizer, layers=layers, replace_tokens=tokens_to_replace)
sample(act_rep, tokenizer, prompt, batch_size=bs, do_sample=True, max_seq_len=256)

  0%|          | 0/206 [00:00<?, ?it/s]

['<s><|user|> Follow the pattern:\ncat X cats is a "plural to singular rule".\nhot Y cold is an "antonym" rule.\nEngland Z London is a "country to capital" rule. \ndog W _____\n\n<|assistant|> dog -> dogs (plural to singular rule)<|end|><|assistant|> dog -> dogs (plural to singular form)\n\nAlternatively:\n\ndog -> puppies (generic/specific pluralization)\n\nNote: While "england -> london" relates to country and capital, "dog" to "dogs" is simply pluralization. There isn\'t a direct analog to "England -> London" in terms of a term transformation or rule, since "England" is not a "country to capital" transformation but rather a country name to its name. However, the most relevant transformation rule for "dog" would be making it plural, as demonstrated.<|end|><|assistant|> However, if we were seeking a similar analogy to the "England to London" transformation, we could say:\n\ndog -> doggie (diminutive/pet form)\n\nThis transformation doesn\'t match a formal "rule" like the ones',
 '<s><|user|> Follow the pattern:\ncat X cats is a "plural to singular rule".\nhot Y cold is an "antonym" rule.\nEngland Z London is a "country to capital" rule. \ndog W ?\n\n<|assistant|> dog -> dogs (plural to singular rule).<|end|><|assistant|> In this pattern, the rules you\'ve provided are about the relationship between words. Following the "plural to singular rule" for "dog," the correct conversion is:\n\ndog -> dog (no change, as it is already the plural form in English)\n\nHowever, if we consider the singular to plural conversion, which might be what\'s usually implied in "plural to singular rule," the answer would be:\n\ndog -> dogs\n\nBut since the original pattern seems to be about direct relationships, "dog -> dogs" doesn\'t fit. Instead, let\'s clarify the pattern as "singular to plural conversion."\n\nGiven this clarification, the answer remains:\n\ndog -> dogs<|end|><|assistant|> dog -> dogs (singular to plural conversion).\n\nBut if we follow the initial pattern\'s intent to find a direct relationship similar to "cat -> c',
 '<s><|user|> Follow the pattern:\ncat X cats is a "plural to singular rule".\nhot Y cold is an "antonym" rule.\nEngland Z London is a "country to capital" rule. \ndog W ____\n\n<|assistant|> dog -> dogs\n\nThis follows the "plural to singular" rule, similar to the example provided with "cat" and "cats." The word "dog" becomes "dogs" when it is plural.<|end|><|assistant|> It\'s important to note that "England" and "London" are related but do not fit the "country to capital" rule as directly as other examples might. "England" refers to the country, and "London" is its capital city. So, for a more precise rule, you could say "country to capital" for "England to London," but the direct transformation in the same pattern as "dog" to "dogs" isn\'t applicable. However, "dog" to "dogs" perfectly fits the provided rule.<|end|><|assistant|> dog -> puppies (in a different context)\n\nAlthough "dog" to "dogs" follows the plural to singular pattern rule, it\'s',
 '<s><|user|> Follow the pattern:\ncat X cats is a "plural to singular rule".\nhot Y cold is an "antonym" rule.\nEngland Z London is a "country to capital" rule. \ndog W ?\n\n<|assistant|> dog -> dogs follows the "plural to singular rule".<|end|><|assistant|> However, if we consider a different kind of transformation, similar to the patterns provided, we could create a new rule. For instance:\n\nDog + "n" (adding an "n") -> Dogs (a morphological change to indicate more than one)\n\nSo in this context, dog doesn\'t transform into antonyms or country to capital transformations, but into a plural form with an added suffix. This is a common rule in English for forming the plural of many nouns.<|end|><|assistant|> Another way to approach this could be:\n\ndog -> puppies (a transformation to a related concept)\n\nDog follows a "noun to related concept" rule, indicating that the noun (dog) is related to its offspring or young ones (puppies).<|end|><|assistant|> Lastly, we could

In [ ]:
jnp.linspace(0, 100, bs)

# jax.Array float32(32,) ≈5e+01 ±3e+01 [≥0.0, ≤1e+02] zero:1 nonzero:31
  Array([  0.       ,   3.2258062,   6.4516125,   9.677419 ,  12.903225 ,
          16.129032 ,  19.354837 ,  22.580643 ,  25.80645  ,  29.032257 ,
          32.258064 ,  35.483868 ,  38.709675 ,  41.935482 ,  45.161285 ,
          48.387093 ,  51.6129   ,  54.838707 ,  58.064514 ,  61.290318 ,
          64.51613  ,  67.74193  ,  70.967735 ,  74.19354  ,  77.41935  ,
          80.64516  ,  83.870964 ,  87.09677  ,  90.32257  ,  93.54838  ,
          96.774185 , 100.       ], dtype=float32)

In [ ]:
from micrlhf.llama import LlamaBlock
from micrlhf.flash import flashify
from micrlhf.sampling import sample, trange, jnp, load_tokenizer, jit_wrapper
get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
    pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_pre.{i}"),
        x
    ])
)
get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
get_resids_call = jit_wrapper.Jitted(get_resids)
def rep_w_linear(mod):
    val = mod.table.embeddings.value  # vocabulary, embedding
    return pz.nn.Linear(pz.nn.Parameter(val, "input_embed"), ["vocabulary"], ["embedding"])
get_resids_one_hot = get_resids.select().at_instances_of(pz.nn.EmbeddingLookup).apply(rep_w_linear)
get_resids_one_hot_call = jit_wrapper.Jitted(get_resids_one_hot)

In [ ]:
from micrlhf.sampling import sample, trange, jnp, load_tokenizer, jit_wrapper
from tqdm.auto import trange
from penzai.toolshed import sharding_util
import dataclasses
from functools import partial
import numpy as np
import jax

@jax.jit
def loss_fn(logits, inputs):
    losses = pz.nx.nmap(lambda l, i: jnp.take_along_axis(jax.nn.log_softmax(l[:-1], -1), i[1:, None], 1)[:, 0].mean()
                        )(logits.untag("seq", "vocabulary"), inputs.tokens.untag("seq"))
    return -losses

bs_start = llama.mesh.shape["dp"]
n_x = 16
# n_x = 19
# n_x = 31
tokens_init = tokenizer.encode(f"<|user|>\nX{' X' * n_x}<|end|>\n<|assistant|>\n")
optim_mask = [token == 1060 for token in tokens_init]
tokens_init = np.asarray(tokens_init)
MAX_ELITES = 4
PROB_SWAP = 0.1  # probability of a swap
PROB_GRADS = 0.8    # probability of using gradients 
tokens_init = np.repeat(tokens_init[None, :], MAX_ELITES, axis=0)
def tokens_to_array(tokens):
    token_array = jnp.asarray(tokens)
    if len(token_array) >= bs_start:
        token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    token_array = pz.nx.wrap(token_array, "batch", "seq")
    return token_array
def run_tokens(token_array, grad_metric=None):
    if not isinstance(token_array, pz.nx.NamedArray):
        token_array = tokens_to_array(token_array)
    inputs = llama.inputs.from_basic_segments(token_array)
    if grad_metric:
        @partial(jax.grad, has_aux=True)
        def lwg(x):
            logits, resids = get_resids_one_hot_call(dataclasses.replace(inputs, tokens=x))
            resids = {resid.tag: resid.value for resid in resids}
            metric = grad_metric(logits, resids, inputs)
            return metric, (logits, resids)
        vocab = llama.select().at_instances_of(pz.nn.EmbeddingLookup).get_sequence()[0].table.embeddings.value.named_shape["vocabulary"]
        one_hots = pz.nx.nmap(lambda x: jax.nn.one_hot(x, vocab))(inputs.tokens).tag("vocabulary")
        grad, (logits, resids) = lwg(one_hots)
    else:
        logits, resids = get_resids_call(inputs)
    losses = loss_fn(logits, inputs)
    if not grad_metric:
        resids = {resid.tag: resid.value for resid in resids}
    return_vals = logits, losses, resids
    if grad_metric:
        return_vals = return_vals + (grad,)
    return return_vals

mask = jax.device_put(jnp.asarray(optim_mask), jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("sp")))

# @partial(jax.jit, static_argnames=("max_inv_temp", "expected_changes"))
@jax.jit
def temper(logits, key, elites, grads, max_inv_temp, expected_changes):
    key_choice, key_random = jax.random.split(key)
    index = jax.random.randint(key_choice, (), 0, len(logits) - 1)
    key_categorical, key_uniform, key_bernoulli, key_randint, key_use_grads, key_mutations = jax.random.split(key_random, 6)
    logit = logits[index]
    elite = elites[index]
    grads = grads[index]
    
    logit = jnp.roll(logit, 1, 0)
    logit = logit * jax.random.uniform(key_uniform, minval=0, maxval=max_inv_temp)
    use_grads = jax.random.bernoulli(key_use_grads, p=PROB_GRADS).astype(jnp.int_)
    logit = jax.lax.switch(use_grads, ((lambda x: x), (lambda x: jnp.where(grads, x, -jnp.inf))), logit)
    # print(expected_changes - 1)
    # print(jnp.maximum)
    to_change = jax.random.bernoulli(key_bernoulli, jnp.maximum(.5, expected_changes - 1) / mask.sum(), mask.shape)
    definite_indices = jax.random.randint(key_randint, mask.shape[:-1], 0, mask.shape[-1])
    definite_mask = jax.nn.one_hot(definite_indices, to_change.shape[-1], dtype=jnp.bool_)
    to_change = to_change | definite_mask
    changed = jnp.where(mask & to_change,
                        jax.random.categorical(key_categorical, logit),
                        elite)
    
    key_swap, key_mutations = jax.random.split(key_mutations)
    swap_indices = jax.random.randint(key_swap, (2,), 0, mask.sum())
    swap_from, swap_to = jnp.nonzero(mask, size=len(mask))[0][swap_indices]
    key_swap, key_mutations = jax.random.split(key_mutations)
    do_swap = jax.random.bernoulli(key_swap, p=PROB_SWAP)
    swapped = changed.at[swap_from].set(changed[swap_to]).at[swap_to].set(changed[swap_from])
    changed = jax.lax.cond(do_swap, lambda x: swapped, lambda x: x, changed)
    
    # key_delete, key_mutations = jax.random.split(key_mutations)
    # delete_index = jax.random.randint(key_delete, (), 0, mask.sum())
    # indices = jnp.cumsum(mask)
    # indices_base = jnp.arange(len(changed))
    # deleted = jnp.where(mask, jnp.where(indices > delete_index, indices_base + 1, indices_base), changed)
    # key_delete, key_mutations = jax.random.split(key_mutations)
    # do_delete = jax.random.bernoulli(key_delete, p=0.1)
    # changed = jax.lax.cond(do_delete, lambda x: deleted, lambda x: x, changed)
    
    return changed

# @partial(jax.jit, static_argnames=("key", "candidates", "expected_changes"))
def algo_iteration(elites, vector, key="resid_pre.16", candidates=128, seed=13, expected_changes=1.5, max_inv_temp=2, topk=128):
    elites = elites.untag("solutions").tag("batch")
    logits, _, _, grads = run_tokens(elites, grad_metric=lambda _l, r, _i: (r[key][{"seq": -1}].untag("embedding") * vector).sum().data_array.mean())
    grads = pz.nx.nmap(lambda x: x >= jax.lax.top_k(x, topk)[0][-1])(grads.untag("vocabulary")).tag("vocabulary")
    logits = logits.untag("batch").tag("elites")

    tempered_samples = pz.nx.nmap(temper)(
        logits.untag("elites", "seq", "vocabulary"),
        pz.nx.wrap(jax.random.split(jax.random.key(seed), candidates), "batch"),
        elites.untag("batch", "seq"),
        grads.untag("batch", "seq", "vocabulary"),
        pz.nx.wrap(jnp.array(max_inv_temp)), pz.nx.wrap(jnp.array(expected_changes))).tag("seq")
    # tempered_samples = sharding_util.name_to_name_device_put(tempered_samples, llama.mesh, dict(batch="dp", seq="sp"))
    _, new_losses, new_resids = run_tokens(tempered_samples)

    new_scores = (new_resids[key][{"seq": -1}].untag("embedding") * vector).sum().astype(new_losses.dtype)
    metrics = pz.nx.nmap(lambda *xs: jnp.stack(xs))(new_losses, new_scores).tag("metrics")
    solution_axes = [k for k in tempered_samples.named_shape.keys() if k != "seq"]
    solutions = tempered_samples.untag(*solution_axes).flatten().tag("solutions").unwrap("solutions", "seq")
    metrics = metrics.untag(*(k for k in solution_axes if k != "seq")).flatten().tag("solutions").unwrap("solutions", "metrics")

    return solutions, metrics


In [ ]:
import random


seed = random.randint(0, 2**32-1)
print("Seed:", seed)
np.random.seed(seed)
toks_init = tokens_init.copy()
toks_init[:, optim_mask] = np.random.randint(100, tokenizer.vocab_size, toks_init[:, optim_mask].shape)
best_metrics = None
best = tokens_to_array(toks_init).untag("batch").tag("solutions")
xent_min = 10
xent_max = 100
weights = jnp.stack((
    jnp.linspace(-xent_max, -xent_min, MAX_ELITES),
    jnp.ones(MAX_ELITES),
), -1)
@partial(jax.jit, donate_argnums=(0, 1))
def combine_solutions(best_metrics, best, metrics, solutions):
    if best_metrics is not None:
        best_metrics = jnp.concatenate((best_metrics, metrics), 0)
        best = pz.nx.nmap(lambda a, b: jnp.concatenate((a, b)))(
            best.untag("solutions"),
            pz.nx.wrap(solutions, "solutions", "seq").untag("solutions")
        ).tag("solutions").unwrap("solutions", "seq")
    else:
        best_metrics = metrics
        best = solutions
    elite_mask = (best_metrics[None, :] * weights[:, None]).sum(-1).argmax(1)
    best_metrics = best_metrics[elite_mask]
    best = pz.nx.wrap(best[elite_mask], "solutions", "seq")
    return best_metrics, best
for seed in (bar := trange(2_500)):
    solutions, metrics = algo_iteration(best, vector, seed=seed)
    best_metrics, best = combine_solutions(best_metrics, best, metrics, solutions)
    m = {}
    for index in range(MAX_ELITES):
        i = index
        m |= {f"decoded.{i}": tokenizer.decode(best[{"solutions": index}].unwrap("seq")),
              f"loss.{i}": best_metrics[index][0], f"score.{i}": best_metrics[index][1]}
    bar.set_postfix(**m)

Seed: 1034250694


  0%|          | 0/2500 [00:00<?, ?it/s]